### Reference

1. https://github.com/ashishpatel26/LLM-Finetuning/blob/main/7.FineTune_LLAMA2_with_QLORA.ipynb  
2. https://huggingface.co/blog/4bit-transformers-bitsandbytes  
3. https://pytorch.org/blog/finetune-llms/  
4. https://colab.research.google.com/drive/1vIjBtePIZwUaHWfjfNHzBjwuXOyU_ugD?usp=sharing#scrollTo=6k_nL6xJMZW2  

# 1. Weight and Bias Login

In [1]:
import wandb
import os
os.environ["WANDB_PROJECT"]="QLoRA_Instruction_finetune_08"

wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: aeolian83. Use `wandb login --relogin` to force relogin


True

# 2. Login Huggingface

In [2]:
from huggingface_hub import login
from dotenv import load_dotenv

load_dotenv()


login(token= os.environ["HF_TOKEN"])

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /home/aeolian83/.cache/huggingface/token
Login successful


# 3. Dataset Load

In [3]:
from datasets import load_dataset

In [4]:
ko_instruction_01 = load_dataset("aeolian83/OpenOrca-gugugo-ko_test", cache_dir="/mnt/t7/.cache/huggingface/datasets")

In [5]:
ko_instruction_01 = ko_instruction_01.shuffle(seed=2160)

In [6]:
ko_instruction_01

DatasetDict({
    train: Dataset({
        features: ['question', 'system_prompt', 'response', 'id'],
        num_rows: 1008132
    })
    test: Dataset({
        features: ['question', 'system_prompt', 'response', 'id'],
        num_rows: 1232162
    })
})

In [7]:
ko_instruction_01 = ko_instruction_01["train"]

In [8]:
ko_instruction_01

Dataset({
    features: ['question', 'system_prompt', 'response', 'id'],
    num_rows: 1008132
})

In [9]:
ko_instruction_01 = ko_instruction_01.train_test_split(test_size=0.2)

In [10]:
ko_instruction_01

DatasetDict({
    train: Dataset({
        features: ['question', 'system_prompt', 'response', 'id'],
        num_rows: 806505
    })
    test: Dataset({
        features: ['question', 'system_prompt', 'response', 'id'],
        num_rows: 201627
    })
})

In [11]:
ko_instruction_01["train"][100]

{'question': "과제 정의와 입력이 주어졌을 때 출력으로 답하세요. 이 과제에서는 자연어로 작성된 5개의 문장으로 구성된 짧은 이야기가 주어집니다. 그러나 주어진 이야기의 순서가 올바르지 않습니다. 주어진 5개의 문장을 올바른 순서로 반환하여 올바른 순서로 새로운 흐름이 있는 일관된 짧은 이야기를 만드는 것이 여러분의 임무입니다. 올바른 순서로 5개의 문장이 주어졌을 때 '23415'와 같이 올바른 순서로 답을 생성하세요.\n\n문장1: 그는 가까운 병원으로 비틀거리며 걸어갔습니다. 문장2: 조는 어느 날 밤 집으로 걸어가고 있었습니다. 문장3: 갑자기 한 남자가 다가와 그를 찌르고 말았습니다. 문장4: 그는 회복하는 데 2주 동안 병원에서 입원했습니다. 문장5: 조는 심하게 다쳤어요!\n",
 'system_prompt': '정의에 어떻게 사용했는지 설명해 주세요.',
 'response': "문장이 일관되게 전개되도록 문장의 올바른 순서는 '25314'입니다. 이 순서는 논리적 순서를 제공합니다: 먼저 조는 어느 날 밤 집으로 걸어가고 있었습니다(문장2), 그 다음에는 한 남자가 다가와 그를 찌르고(문장3), 그 결과 그는 심하게 다쳤습니다(문장5). 그 후 그는 가까운 병원으로 비틀거리며 걸어갔고(문장1), 회복을 위해 2주 동안 그곳에 있었습니다(문장4).",
 'id': 'niv.379769'}

In [12]:
dataset_name = "aeolian83/OpenOrca-gugugo-ko_test"

# 4. Loading the Model

In [13]:
model_id = "beomi/llama-2-ko-7b"
device_map = {"": 0}
cache_model_dir="/mnt/t7/.cache/huggingface/models"

In [14]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

In [15]:
# Settings for 4-bit QLoRA Training(4bit QLoRA 학습을 위한 설정)
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True, 
    bnb_4bit_compute_dtype=torch.bfloat16, # Nvidia의 Ampere 아키텍처 이후 가속기는 bf16으로 속도 향상을 꾀할수 있다. 
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)

# bnb_4bit_quant_type="nf4" 설정상 기본값은 bnb_4bit_quant_type="fp4"이나 허깅페이스 저자들에 의하면
# 경험적 결과로 "nf4"가 결과가 더 좋았다고 한다. https://huggingface.co/blog/4bit-transformers-bitsandbytes
# bnb_4bit_use_double_quant=True로 하면 매개변수단 0.4bit을 추가로 절약 할 수 있다고 한다. 

In [16]:
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=quantization_config, device_map=device_map, cache_dir=cache_model_dir, trust_remote_code=True)
model.config.use_cache = False

# model.config.pretraining_tp = 1
# 종종 QLoRA 코드에 이 코드가 보이는데 병렬 학습에 쓰이는 코드로 보인다. 

Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

In [17]:
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True, cache_dir=cache_model_dir)
tokenizer.add_special_tokens({'pad_token': '<PAD>'})



1

In [18]:
# 이 코드를 쓰지 않는 경우(물론 패딩 토큰을 별도로 사용하는 경우에 해당됨) loss가 0으로 떨어지는 경우가 있다함
tokenizer.padding_side = "right"

In [19]:
model.resize_token_embeddings(len(tokenizer)) # pad_token이 추가되었으므로 embedding과 language modeling head를 resize

Embedding(46337, 4096)

# 5. LoRA Setup

In [20]:
from peft import LoraConfig, get_peft_model

lora_alpha = 16
lora_dropout = 0.1
lora_r = 64

In [21]:
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM"
)

# 6. Formatting Dataset

In [22]:
def format_instruction(sample):
    system_prompt = f"### system prompt: {sample['system_prompt']}"
    input = f"### question: {sample['question']}" if len(sample["question"]) > 0 else None
    output = f"### response: {sample['response']}"
    # join all the parts together
    prompt = "\n\n".join([i for i in [system_prompt, input, output] if i is not None])
    return prompt

# template dataset to add prompt to each sample
def template_dataset(sample):
    sample["text"] = f"{format_instruction(sample)}{tokenizer.eos_token}"
    return sample

In [23]:
train_dataset = ko_instruction_01['train'].map(template_dataset, remove_columns=list(ko_instruction_01['train'].features), num_proc=10)

Map (num_proc=10):   0%|          | 0/806505 [00:00<?, ? examples/s]

In [24]:
train_dataset["text"][100]

"### system prompt: 정의에 어떻게 사용했는지 설명해 주세요.\n\n### question: 과제 정의와 입력이 주어졌을 때 출력으로 답하세요. 이 과제에서는 자연어로 작성된 5개의 문장으로 구성된 짧은 이야기가 주어집니다. 그러나 주어진 이야기의 순서가 올바르지 않습니다. 주어진 5개의 문장을 올바른 순서로 반환하여 올바른 순서로 새로운 흐름이 있는 일관된 짧은 이야기를 만드는 것이 여러분의 임무입니다. 올바른 순서로 5개의 문장이 주어졌을 때 '23415'와 같이 올바른 순서로 답을 생성하세요.\n\n문장1: 그는 가까운 병원으로 비틀거리며 걸어갔습니다. 문장2: 조는 어느 날 밤 집으로 걸어가고 있었습니다. 문장3: 갑자기 한 남자가 다가와 그를 찌르고 말았습니다. 문장4: 그는 회복하는 데 2주 동안 병원에서 입원했습니다. 문장5: 조는 심하게 다쳤어요!\n\n\n### response: 문장이 일관되게 전개되도록 문장의 올바른 순서는 '25314'입니다. 이 순서는 논리적 순서를 제공합니다: 먼저 조는 어느 날 밤 집으로 걸어가고 있었습니다(문장2), 그 다음에는 한 남자가 다가와 그를 찌르고(문장3), 그 결과 그는 심하게 다쳤습니다(문장5). 그 후 그는 가까운 병원으로 비틀거리며 걸어갔고(문장1), 회복을 위해 2주 동안 그곳에 있었습니다(문장4).</s>"

# 7. Training Argument Setup

In [25]:
from transformers import TrainingArguments

In [26]:
checkpoint_dir = "./checkpoint/gugugo-experi_01"

In [27]:
output_dir = checkpoint_dir
per_device_train_batch_size = 1
gradient_accumulation_steps = 2
optim = "paged_adamw_32bit"
report_to="wandb"
save_steps = 500
save_total_limit=5
num_train_epochs = 0.9
logging_steps = 20
learning_rate = 2e-4
max_grad_norm = 0.3
warmup_ratio = 0.03
lr_scheduler_type = "linear"

In [28]:
training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    num_train_epochs=num_train_epochs,
    save_steps=save_steps,
    save_total_limit=save_total_limit,
    logging_steps=logging_steps,
    report_to = report_to,
    learning_rate=learning_rate,
    bf16=True,
    max_grad_norm=max_grad_norm,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
)

In [29]:
from trl import SFTTrainer

max_seq_length = 1024

In [30]:
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    # max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)

/home/aeolian83/anaconda3/envs/llm_for_p311/lib/python3.11/site-packages/trl/trainer/sft_trainer.py:245: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/806505 [00:00<?, ? examples/s]

/home/aeolian83/anaconda3/envs/llm_for_p311/lib/python3.11/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [31]:
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

# 8. Training

In [32]:
# trainer.train()

  0%|          | 0/362927 [00:00<?, ?it/s]

{'loss': 2.9643, 'grad_norm': 0.11669921875, 'learning_rate': 3.6737692872887585e-07, 'epoch': 0.0}
{'loss': 3.5213, 'grad_norm': 0.1728515625, 'learning_rate': 7.347538574577517e-07, 'epoch': 0.0}
{'loss': 3.5268, 'grad_norm': 0.107421875, 'learning_rate': 1.1021307861866277e-06, 'epoch': 0.0}
{'loss': 3.2377, 'grad_norm': 0.1513671875, 'learning_rate': 1.4695077149155034e-06, 'epoch': 0.0}
{'loss': 3.9255, 'grad_norm': 0.2294921875, 'learning_rate': 1.8368846436443793e-06, 'epoch': 0.0}
{'loss': 3.0301, 'grad_norm': 0.1513671875, 'learning_rate': 2.2042615723732554e-06, 'epoch': 0.0}
{'loss': 3.4576, 'grad_norm': 0.185546875, 'learning_rate': 2.571638501102131e-06, 'epoch': 0.0}
{'loss': 3.6152, 'grad_norm': 0.126953125, 'learning_rate': 2.9390154298310068e-06, 'epoch': 0.0}
{'loss': 3.1777, 'grad_norm': 0.1455078125, 'learning_rate': 3.3063923585598822e-06, 'epoch': 0.0}
{'loss': 4.0159, 'grad_norm': 0.361328125, 'learning_rate': 3.6737692872887586e-06, 'epoch': 0.0}
{'loss': 2.9886

/home/aeolian83/anaconda3/envs/llm_for_p311/lib/python3.11/site-packages/peft/utils/save_and_load.py:168: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


{'loss': 2.4219, 'grad_norm': 0.51171875, 'learning_rate': 9.551800146950773e-06, 'epoch': 0.0}
{'loss': 2.6931, 'grad_norm': 0.88671875, 'learning_rate': 9.919177075679648e-06, 'epoch': 0.0}
{'loss': 2.5441, 'grad_norm': 0.369140625, 'learning_rate': 1.0286554004408523e-05, 'epoch': 0.0}
{'loss': 2.3462, 'grad_norm': 0.66796875, 'learning_rate': 1.06539309331374e-05, 'epoch': 0.0}
{'loss': 2.4817, 'grad_norm': 1.0859375, 'learning_rate': 1.1021307861866276e-05, 'epoch': 0.0}
{'loss': 2.1198, 'grad_norm': 0.400390625, 'learning_rate': 1.1388684790595152e-05, 'epoch': 0.0}
{'loss': 2.2072, 'grad_norm': 0.81640625, 'learning_rate': 1.1756061719324027e-05, 'epoch': 0.0}
{'loss': 2.1549, 'grad_norm': 0.515625, 'learning_rate': 1.2123438648052903e-05, 'epoch': 0.0}
{'loss': 2.0403, 'grad_norm': 0.482421875, 'learning_rate': 1.2490815576781778e-05, 'epoch': 0.0}
{'loss': 2.1956, 'grad_norm': 1.6015625, 'learning_rate': 1.2858192505510655e-05, 'epoch': 0.0}
{'loss': 1.9375, 'grad_norm': 0.542

/home/aeolian83/anaconda3/envs/llm_for_p311/lib/python3.11/site-packages/peft/utils/save_and_load.py:168: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


...

{'loss': 1.2993, 'grad_norm': 0.4765625, 'learning_rate': 2.4092217055496693e-05, 'epoch': 0.79}
{'loss': 1.1599, 'grad_norm': 0.84765625, 'learning_rate': 2.408085467803283e-05, 'epoch': 0.79}
{'loss': 1.2235, 'grad_norm': 0.6015625, 'learning_rate': 2.4069492300568972e-05, 'epoch': 0.79}
{'loss': 1.1969, 'grad_norm': 0.875, 'learning_rate': 2.405812992310511e-05, 'epoch': 0.79}
{'loss': 1.0034, 'grad_norm': 0.70703125, 'learning_rate': 2.404676754564125e-05, 'epoch': 0.8}
{'loss': 1.3536, 'grad_norm': 0.609375, 'learning_rate': 2.4035405168177393e-05, 'epoch': 0.8}
{'loss': 1.1303, 'grad_norm': 0.8828125, 'learning_rate': 2.402404279071353e-05, 'epoch': 0.8}
{'loss': 1.3232, 'grad_norm': 0.6484375, 'learning_rate': 2.401268041324967e-05, 'epoch': 0.8}
{'loss': 1.2824, 'grad_norm': 0.68359375, 'learning_rate': 2.4001318035785806e-05, 'epoch': 0.8}
{'loss': 1.0439, 'grad_norm': 0.9453125, 'learning_rate': 2.398995565832195e-05, 'epoch': 0.8}
{'loss': 1.3491, 'grad_norm': 0.66015625, 'l

/home/aeolian83/anaconda3/envs/llm_for_p311/lib/python3.11/site-packages/peft/utils/save_and_load.py:168: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


{'loss': 1.4636, 'grad_norm': 0.5, 'learning_rate': 2.3808157618900177e-05, 'epoch': 0.8}
{'loss': 1.1827, 'grad_norm': 0.671875, 'learning_rate': 2.379679524143632e-05, 'epoch': 0.8}
{'loss': 1.2326, 'grad_norm': 0.47265625, 'learning_rate': 2.378543286397246e-05, 'epoch': 0.8}
{'loss': 1.3258, 'grad_norm': 0.7109375, 'learning_rate': 2.3774070486508597e-05, 'epoch': 0.8}
{'loss': 0.9867, 'grad_norm': 0.828125, 'learning_rate': 2.376270810904474e-05, 'epoch': 0.8}
{'loss': 1.3248, 'grad_norm': 0.5703125, 'learning_rate': 2.3751345731580876e-05, 'epoch': 0.8}
{'loss': 1.2427, 'grad_norm': 0.78515625, 'learning_rate': 2.3739983354117014e-05, 'epoch': 0.8}
{'loss': 1.2135, 'grad_norm': 0.4453125, 'learning_rate': 2.3728620976653156e-05, 'epoch': 0.8}
{'loss': 1.1848, 'grad_norm': 0.71875, 'learning_rate': 2.3717258599189297e-05, 'epoch': 0.8}
{'loss': 1.0303, 'grad_norm': 0.7734375, 'learning_rate': 2.3705896221725435e-05, 'epoch': 0.8}
{'loss': 1.2875, 'grad_norm': 0.92578125, 'learning

/home/aeolian83/anaconda3/envs/llm_for_p311/lib/python3.11/site-packages/peft/utils/save_and_load.py:168: UserWarning: Setting `save_embedding_layers` to `True` as the embedding layer has been resized during finetuning.
  warnings.warn(


{'loss': 1.2276, 'grad_norm': 0.609375, 'learning_rate': 2.3524098182303667e-05, 'epoch': 0.8}
{'loss': 1.0768, 'grad_norm': 0.796875, 'learning_rate': 2.3512735804839805e-05, 'epoch': 0.8}
{'loss': 1.2469, 'grad_norm': 0.48828125, 'learning_rate': 2.3501373427375943e-05, 'epoch': 0.8}
{'loss': 1.4802, 'grad_norm': 0.7734375, 'learning_rate': 2.3490011049912084e-05, 'epoch': 0.8}


In [ ]:
trainer.train(resume_from_checkpoint=True)

In [ ]:
lora_model_save_dir = "./results/gugugo-experi_01"

In [ ]:
model_to_save = trainer.model.module if hasattr(trainer.model, 'module') else trainer.model  # Take care of distributed/parallel training
model_to_save.save_pretrained(lora_model_save_dir, save_embedding_layers = True)

In [ ]:
lora_config = LoraConfig.from_pretrained(lora_model_save_dir)
model = get_peft_model(model, lora_config)

In [ ]:
tokenizer.push_to_hub('aeolian83/llama_ko_sft_gugugo_experi_01')

CommitInfo(commit_url='https://huggingface.co/aeolian83/llama_ko_sft_kullm_experi_05/commit/a660bf40db1d86c7dea85c0a2328f73c61f61596', commit_message='Upload tokenizer', commit_description='', oid='a660bf40db1d86c7dea85c0a2328f73c61f61596', pr_url=None, pr_revision=None, pr_num=None)